# AutoGluon Tabular with SageMaker

<br>

## Prerequisites

If using a SageMaker hosted notebook, select kernel `conda_mxnet_p36`.

In [1]:
# Make sure docker compose is set up properly for local mode
!./setup.sh

The user has root access.
SageMaker instance route table setup is ok. We are good to go.
SageMaker instance routing for Docker is ok. We are good to go!


In [2]:
# Used to perform df.to_csv('s3://<path>')
!pip install pandas==1.0.3 s3fs==0.4.0 | grep -v 'already satisfied'

  Found existing installation: pandas 0.24.2
    Uninstalling pandas-0.24.2:
      Successfully uninstalled pandas-0.24.2
  Found existing installation: s3fs 0.1.5
    Uninstalling s3fs-0.1.5:
      Successfully uninstalled s3fs-0.1.5
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
# Imports
import os
import boto3
import sagemaker
from collections import Counter
import pandas as pd
from sagemaker import get_execution_role, local, Model
from sagemaker.estimator import Estimator
from sagemaker.predictor import RealTimePredictor, csv_serializer, json_deserializer
from sklearn.metrics import accuracy_score, classification_report
from IPython.core.display import display, HTML
from IPython.core.interactiveshell import InteractiveShell

# Print settings
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 10)

# Account/s3 setup
session = sagemaker.Session()
local_session = local.LocalSession()
bucket = session.default_bucket()
prefix = 'sagemaker/autogluon-tabular'
region = session.boto_region_name
role = get_execution_role()
client = boto3.client('sts')
account = client.get_caller_identity()['Account']

### Build training docker image

First, build autogluon package to copy into docker image

In [4]:
if not os.path.exists('package'):
    !pip install PrettyTable -t package
    !pip install bokeh -t package
    !pip install --pre autogluon -t package
    !pip install numpy==1.16.1 -t package    
    !pip install --upgrade boto3 -t package
    !pip install bokeh -t package
    !pip install --upgrade matplotlib -t package

  Running setup.py bdist_wheel for PrettyTable ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/80/34/1c/3967380d9676d162cb59513bd9dc862d0584e045a162095606
Successfully built PrettyTable
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 8.6MB 6.0MB/s eta 0:00:01
    100% |████████████████████████████████| 235kB 38.9MB/s ta 0:00:01
    100% |████████████████████████████████| 133kB 43.1MB/s ta 0:00:01
    100% |████████████████████████████████| 20.2MB 2.5MB/s eta 0:00:01
    100% |████████████████████████████████| 2.1MB 9.6MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/62/0a/34641d2bf5c917c96db0ded85ae4da25b6cd922d6b794648d4e7e07c88e5/packaging-20.3-py2.py3-none-any.whl
    100% |████████████████████████████████| 501kB 33.0MB/s ta 0:00:01    90% |█████████████████████████████   | 450kB 59.9MB/s eta 0:00:01
  U

    100% |████████████████████████████████| 133kB 42.3MB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/06/1e/1e08baaaf6c3d3df1459fd85f0e7d2d6aa916f33958f151ee1ecc9800971/paramiko-2.7.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/07/08/a549ba8b061005bb629b76adc000f3caaaf881028b963c2e18f811c6edc1/numpy-1.18.2-cp36-cp36m-manylinux1_x86_64.whl
    100% |████████████████████████████████| 7.1MB 11.1MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 33.3MB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/45/73/d18a8884de8bffdcda475728008b5b13be7fbef40a2acc81a0d5d524175d/cryptography-2.8-cp34-abi3-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/c4/b8/3512f0e93e0db23a71d82485ba256071ebef99b227351f0f5540f744af41/psutil-5.7.0.tar.gz
    100% |████████████████████████████████| 12.4MB 4.4MB/s eta 0:00:01
    100% |████████████████████████████████| 10.4MB 8.0MB/s eta 0:00:01
    100% |

    100% |████████████████████████████████| 399kB 38.6MB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/d4/70/d60450c3dd48ef87586924207ae8907090de0b306af2bce5d134d78615cb/python_dateutil-2.8.1-py2.py3-none-any.whl
    100% |████████████████████████████████| 92kB 38.2MB/s ta 0:00:01
    100% |████████████████████████████████| 512kB 38.4MB/s ta 0:00:01
    100% |████████████████████████████████| 92kB 40.6MB/s ta 0:00:01
    100% |████████████████████████████████| 51kB 20.8MB/s ta 0:00:01
    100% |████████████████████████████████| 276kB 37.4MB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/22/cd/a6aa959dca619918ccb55023b4cb151949c64d4d5d55b3f4ffd7eee0c6e8/docutils-0.15.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ae/e7/d9c3a176ca4b02024debf82342dab36efadfc5776f9c8db077e8f6e71821/pycparser-2.20-py2.py3-none-any.whl
    100% |████████████████████████████████| 583kB 29.4MB/s ta 0:00:01
  Running setup.py bdist_wheel for 

Target directory /home/ec2-user/SageMaker/autogluon-tabular-sagemaker/package/dateutil already exists. Specify --upgrade to force replacement.
Target directory /home/ec2-user/SageMaker/autogluon-tabular-sagemaker/package/numpy-1.18.2.dist-info already exists. Specify --upgrade to force replacement.
Target directory /home/ec2-user/SageMaker/autogluon-tabular-sagemaker/package/tornado already exists. Specify --upgrade to force replacement.
Target directory /home/ec2-user/SageMaker/autogluon-tabular-sagemaker/package/pyparsing-2.4.6.dist-info already exists. Specify --upgrade to force replacement.
Target directory /home/ec2-user/SageMaker/autogluon-tabular-sagemaker/package/numpy already exists. Specify --upgrade to force replacement.
Target directory /home/ec2-user/SageMaker/autogluon-tabular-sagemaker/package/__pycache__ already exists. Specify --upgrade to force replacement.
Target directory /home/ec2-user/SageMaker/autogluon-tabular-sagemaker/package/python_dateutil-2.8.1.dist-info al

You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
  Using cached https://files.pythonhosted.org/packages/93/4b/52da6b1523d5139d04e02d9e26ceda6146b48f2a4e5d2abfdf1c7bac8c40/matplotlib-3.2.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/d4/70/d60450c3dd48ef87586924207ae8907090de0b306af2bce5d134d78615cb/python_dateutil-2.8.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/5d/bc/1e58593167fade7b544bfe9502a26dc860940a79ab306e651e7f13be68c2/pyparsing-2.4.6-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f7/d2/e07d3ebb2bd7af696440ce7e754c59dd546ffe1bbe732c8ab68b9c834e61/cycler-0.10.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/07/08/a549ba8b061005bb629b76adc000f3caaaf881028b963c2e18f811c6edc1/numpy-1.18.2-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pyth

Now build the training image and push to ECR

In [5]:
training_algorithm_name = 'autogluon-sagemaker-training'

In [36]:
!./container-training/build_push_training.sh {training_algorithm_name}

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon  1.001GB
Step 1/8 : FROM 763104351884.dkr.ecr.us-east-2.amazonaws.com/mxnet-training:1.6.0-cpu-py3
 ---> 129a8893d865
Step 2/8 : RUN pip install --upgrade pip
 ---> Using cache
 ---> d94b2ee91d31
Step 3/8 : ENV PATH="/opt/ml/code:${PATH}"
 ---> Using cache
 ---> 40261fdfe01e
Step 4/8 : COPY package/ /opt/ml/code/package/
 ---> Us

### Get the data

In this example we'll use the direct-marketing dataset to build a binary classification model that predicts whether customers will accept or decline a marketing offer.  
First we'll download the data and split it into train and test sets. AutoGluon does not require a separate validation set (it uses bagged k-fold cross-validation).

In [37]:
# Download and unzip the data
!wget -N https://sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com/autopilot/direct_marketing/bank-additional.zip --quiet
!unzip -qq -o bank-additional.zip
!rm bank-additional.zip

local_data_path = './bank-additional/bank-additional-full.csv'
data = pd.read_csv(local_data_path)

# Split train/test data
train = data.sample(frac=0.9, random_state=42)
test = data.drop(train.index)

# Split test X/y
target = 'y'
y_test = test[target]
X_test = test.drop(columns=[target])

##### Check the data

In [38]:
train.head(3)
train.shape

test.head(3)
test.shape

X_test.head(3)
X_test.shape

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
32884,57,technician,married,high.school,no,no,yes,cellular,may,mon,371,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
3169,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,285,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
32206,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,52,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no


(37069, 21)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
9,25,services,single,high.school,no,yes,no,telephone,may,mon,50,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
55,55,technician,married,professional.course,unknown,yes,no,telephone,may,mon,135,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
60,47,entrepreneur,married,university.degree,unknown,no,no,telephone,may,mon,449,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


(4119, 21)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
9,25,services,single,high.school,no,yes,no,telephone,may,mon,50,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0
55,55,technician,married,professional.course,unknown,yes,no,telephone,may,mon,135,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0
60,47,entrepreneur,married,university.degree,unknown,no,no,telephone,may,mon,449,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0


(4119, 20)

Upload the data to s3

In [9]:
# Set s3 paths
train_s3_path = f's3://{bucket}/{prefix}/data/train.csv'
test_s3_path = f's3://{bucket}/{prefix}/data/test.csv'
X_test_s3_path = f's3://{bucket}/{prefix}/data/X_test.csv'

# Upload
train.to_csv(train_s3_path, index=False)
print(f'Train data uploaded to {train_s3_path}')
test.to_csv(test_s3_path, index=False)
print(f'Labeled Test data uploaded to {test_s3_path}')
X_test.to_csv(X_test_s3_path, index=False)
print(f'Unlabeled Test data uploaded to {X_test_s3_path}')

Train data uploaded to s3://sagemaker-us-east-2-332270294944/sagemaker/autogluon-tabular/data/train.csv
Labeled Test data uploaded to s3://sagemaker-us-east-2-332270294944/sagemaker/autogluon-tabular/data/test.csv
Unlabeled Test data uploaded to s3://sagemaker-us-east-2-332270294944/sagemaker/autogluon-tabular/data/X_test.csv


## Train

For local training set `train_instance_type` to `local` .  
For non-local training the recommended instance type is `ml.m5.24xlarge` .

In [39]:
%%time

instance_type = 'ml.m5.24xlarge'
#instance_type = 'local'

ecr_image = f'{account}.dkr.ecr.{region}.amazonaws.com/{training_algorithm_name}:latest'

# Example autogluon hyperparameters
model_hps = {
    'NN': {'num_epochs': '500'},
    'GBM': {'num_boost_round': '10000',
            'num_leaves': 'ag.space.Int(lower=26, upper=66, default=36)'},
    'CAT': {'iterations': '10000'}, 
    'KNN': {},
    # (Adding RF or ET increase model export time)
}

hyperparameters = {
    'target': 'y',
    'time_limit': 2*60,
    'auto_stack': True,
    'num_bagging_sets': 20,
    'search_strategy': 'random',
    'hyperparameters': model_hps
}

estimator = Estimator(image_name=ecr_image,
                      role=role,
                      train_instance_count=1,
                      train_instance_type=instance_type,
                      hyperparameters=hyperparameters)

estimator.fit(train_s3_path)

2020-03-24 10:19:07 Starting - Starting the training job...
2020-03-24 10:19:08 Starting - Launching requested ML instances......
2020-03-24 10:20:33 Starting - Preparing the instances for training...
2020-03-24 10:21:08 Downloading - Downloading input data...
2020-03-24 10:21:13 Training - Downloading the training image....2020-03-24 10:22:12,531 sagemaker-containers INFO     Imported framework sagemaker_mxnet_container.training
2020-03-24 10:22:12,532 sagemaker-containers INFO     Failed to parse hyperparameter search_strategy value random to Json.
Returning the value itself
2020-03-24 10:22:12,532 sagemaker-containers INFO     Failed to parse hyperparameter auto_stack value True to Json.
Returning the value itself
2020-03-24 10:22:12,532 sagemaker-containers INFO     Failed to parse hyperparameter hyperparameters value {'NN': {'num_epochs': '500'}, 'GBM': {'num_boost_round': '10000', 'num_leaves': 'ag.space.Int(lower=26, upper=66, default=36)'}, 'CAT': {'iterations': '10000'}, 'KNN'


2020-03-24 10:22:11 Training - Training image download completed. Training in progress./usr/local/lib/python3.6/site-packages/mxnet/optimizer/optimizer.py:167: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  Optimizer.opt_registry[name].__name__))
package/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)

====== args ======
hosts,  type: <class 'list'>,  value: ['algo-1']
current_host,  type: <class 'str'>,  value: algo-1
num_gpus,  type: <class 'int'>,  value: 0
model_dir,  type: <class 'str'>,  value: /opt/ml/model
train,  type: <class 'str'>,  value: /opt/ml/input/data/training
target

Attempting to fit model without HPO, but search space is provided. fit() will only consider default hyperparameter values from search space.
Attempting to fit model without HPO, but search space is provided. fit() will only consider default hyperparameter values from search space.
Attempting to fit model without HPO, but search space is provided. fit() will only consider default hyperparameter values from search space.
Attempting to fit model without HPO, but search space is provided. fit() will only consider default hyperparameter values from search space.
Attempting to fit model without HPO, but search space is provided. fit() will only consider default hyperparameter values from search space.
#0110.9207#011 = Validation accuracy score
#0119.63s#011 = Training runtime
#0110.16s#011 = Validation runtime
Fitting model: CatboostClassifier_STACKER_l1 ... Training model for up to 58.95s of the 58.95s of remaining time.

2020-03-24 10:24:39 Uploading - Uploading generated training model#01

In [12]:
# Optionally, download model_data and display training performance graph
!rm -rf model_data && mkdir model_data && aws s3 cp {estimator.model_data} - | tar -xz -C model_data
d(HTML('model_data/SummaryOfModels.html'))

### Build inference docker image

Build the inference image and push to ECR

In [27]:
inference_algorithm_name = 'autogluon-sagemaker-inference'

In [28]:
!./container-inference/build_push_inference.sh {inference_algorithm_name}

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon  1.001GB
Step 1/4 : FROM 763104351884.dkr.ecr.us-east-2.amazonaws.com/mxnet-inference:1.6.0-cpu-py3
 ---> 1e1193fbea5c
Step 2/4 : RUN pip install --upgrade pip
 ---> Using cache
 ---> 28fe56fd6a2d
Step 3/4 : COPY package/ /opt/ml/code/package/
 ---> Using cache
 ---> 1c40a4299495
Step 4/4 : ENV SAGEMAKER_PROGRAM inference.py
 ---

### Create Model

In [40]:
# Create predictor object
class AutoGluonTabularPredictor(RealTimePredictor):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, content_type='text/csv', 
                         serializer=csv_serializer, 
                         deserializer=csv_deserializer, **kwargs)

In [41]:
ecr_image = f'{account}.dkr.ecr.{region}.amazonaws.com/{inference_algorithm_name}:latest'

if instance_type == 'local':
    model = estimator.create_model(image=ecr_image, role=role)
else:
    model_uri = os.path.join(estimator.output_path, estimator._current_job_name, "output", "model.tar.gz")
    model = Model(model_uri, ecr_image, role=role, sagemaker_session=session, predictor_cls=AutoGluonTabularPredictor)

### Batch Transform

For local mode, either `s3://<bucket>/<prefix>/output/` or `file:///<absolute_local_path>` can be used as outputs.

Include label column in input data to also evaluate prediction performance (In this case, passing `test_s3_path` instead of `X_test_s3_path`).

In [42]:
output_path = f's3://{bucket}/{prefix}/output/'
# output_path = f'file://{os.getcwd()}'

transformer = model.transformer(instance_count=1, 
                                instance_type=instance_type,
                                strategy='SingleRecord',
                                assemble_with='Line',
                                max_payload=1,
                                max_concurrent_transforms=100,                              
                                output_path=output_path)

transformer.transform(test_s3_path, content_type='text/csv')
transformer.wait()

...................2020-03-24 10:28:19,796 [INFO ] main com.amazonaws.ml.mms.ModelServer - 
MMS Home: /usr/local/lib/python3.6/site-packages
Current directory: /
Temp directory: /home/model-server/tmp
Number of GPUs: 0
Number of CPUs: 96
Max heap size: 27305 M
Python executable: /usr/local/bin/python3.6
Config file: /etc/sagemaker-mms.properties
Inference address: http://0.0.0.0:8080
Management address: http://0.0.0.0:8080
Model Store: /.sagemaker/mms/models
Initial Models: ALL
Log dir: /logs
Metrics dir: /logs
Netty threads: 0
Netty client threads: 0
Default workers per model: 96
Blacklist Regex: N/A
Maximum Response Size: 6553500
Maximum Request Size: 6553500
2020-03-24 10:28:19,834 [INFO ] main com.amazonaws.ml.mms.wlm.ModelManager - Model model loaded.
2020-03-24 10:28:19,958 [INFO ] main com.amazonaws.ml.mms.ModelServer - Initialize Inference server with: EpollServerSocketChannel.
2020-03-24 10:28:20,148 [INFO ] main com.amazonaws.ml.mms.ModelServer - Inference API bind to: http:/

2020-03-24T10:28:27.272:[sagemaker logs]: MaxConcurrentTransforms=100, MaxPayloadInMB=1, BatchStrategy=SINGLE_RECORD
2020-03-24 10:28:27,260 [INFO ] pool-1-thread-97 ACCESS_LOG - /169.254.255.130:38348 "GET /ping HTTP/1.1" 200 11
2020-03-24 10:28:27,267 [INFO ] epollEventLoopGroup-3-2 ACCESS_LOG - /169.254.255.130:38352 "GET /execution-parameters HTTP/1.1" 404 0
2020-03-24 10:28:27,375 [INFO ] W-9030-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Loading model from /opt/ml/model with contents ['code', 'learner.pkl', 'models', 'SummaryOfModels.html']
2020-03-24 10:28:27,260 [INFO ] pool-1-thread-97 ACCESS_LOG - /169.254.255.130:38348 "GET /ping HTTP/1.1" 200 11
2020-03-24 10:28:27,267 [INFO ] epollEventLoopGroup-3-2 ACCESS_LOG - /169.254.255.130:38352 "GET /execution-parameters HTTP/1.1" 404 0
2020-03-24 10:28:27,375 [INFO ] W-9030-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Loading model from /opt/ml/model with contents ['code', 'learner.pkl', 'models', 'SummaryOfM

In [ ]:
# Check s3 for <filename>.csv.out file
if instance_type != 'local':
    !aws s3 ls {transformer.output_path} --recursive
elif 's3' in output_path:
    !aws s3 ls {output_path+transformer.latest_transform_job.job_name} --recursive

### Endpoint

##### Deploy remote or local endpoint

In [ ]:
instance_type = 'ml.m5.24xlarge'
instance_type = 'local'

predictor = model.deploy(initial_instance_count=1, 
                         instance_type=instance_type)

##### Attach to endpoint (or reattach if kernel was restarted)

In [ ]:
# Select standard or local session based on instance_type
if instance_type == 'local': sess = local_session
else: sess = session

# Attach to endpoint
predictor = AutoGluonTabularPredictor(predictor.endpoint, sagemaker_session=sess)

##### Predict on unlabeled test data

In [ ]:
results = predictor.predict(X_test.to_csv())

# Double check output
print(results)

##### Predict on data that includes label column  
Prediction performance metrics will be printed to endpoint logs.

In [ ]:
results = predictor.predict(test.to_csv())

# Double check output
Counter(results)

##### Check that performance metrics match evaluation printed to endpoint logs as expected

In [ ]:
print(accuracy_score(y_true=y_test, y_pred=results))

print(classification_report(y_true=y_test, y_pred=results, digits=6))

##### Clean up endpoint

In [ ]:
predictor.delete_endpoint()

<br><br><br><br>

# FAQ

- **Q**: I get a disk full error when running local training and deployment, but my disk isn't full.
    - **A**: SageMaker uses `/tmp` folder which has a max size of ~10GB on SM hosted notebooks. Run `sudo rm -rf /tmp/*` to clean.    
- **Q**: I'm having docker issues with local training or deployment, what can I do?
    - **A**: Try deleting running containers. For example: `docker stop $(docker ps -aq) && docker rm $(docker ps -aq)`